In [1]:
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from keras.layers.core import SpatialDropout2D, Permute, Activation, Reshape
from keras.layers.merge import add, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.engine.topology import Input
from keras.models import Model

Using TensorFlow backend.


In [10]:
inp = Input(shape=(256, 256, 3))
conv = Conv2D(13, (3, 3), padding='same', strides=(2, 2))(inp)

## How come max pooling does not rduce borders
max_pool = MaxPooling2D(padding='valid')(inp)

##
merged = concatenate([conv, max_pool], axis=3)
model = Model(inp, merged)


In [11]:
model.output_shape

(None, 128, 128, 16)

In [12]:

enet = BatchNormalization(momentum=0.1)(merged)  # enet_unpooling uses momentum of 0.1, keras default is 0.99
enet = PReLU(shared_axes=[1, 2])(enet)

In [17]:
encoder = Conv2D(16, (2, 2),
                 # padding='same',
                 strides=(2, 2), use_bias=False)(enet)
# Batch normalization + PReLU
encoder = BatchNormalization(momentum=0.1)(encoder)  # enet uses momentum of 0.1, keras default is 0.99
encoder = PReLU(shared_axes=[1, 2])(encoder)
encoder = Conv2D(16, (3, 3), padding='same')(encoder)
encoder = BatchNormalization(momentum=0.1)(encoder)  # enet uses momentum of 0.1, keras default is 0.99
encoder = PReLU(shared_axes=[1, 2])(encoder)
encoder = Conv2D(64, (1, 1), use_bias=False)(encoder)

encoder = BatchNormalization(momentum=0.1)(encoder)  # enet uses momentum of 0.1, keras default is 0.99
encoder = SpatialDropout2D(0.01)(encoder)

In [18]:
model = Model(inp, encoder)
model.output_shape

(None, 64, 64, 64)